In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [2]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from tensorflow.python.client import device_lib

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
print("TensorFlow Version:", tf.__version__)
#print("GPU Available:", tf.test.is_gpu_available())
print("GPU Available:", tf.config.list_physical_devices('GPU'))

tf.keras.backend.clear_session()  # For easy reset of notebook state.

TensorFlow Version: 2.9.1
GPU Available: []


In [5]:
import tensorflow.python.util.deprecation as deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

In [6]:
!python3 setup.py build_ext --inplace  

running build_ext


In [10]:
!python3 pre_entrenamiento.py --path '.' --out_name 'prueba_cutoff_3_maxneigh_250' --cutoff 3.0 --max_neigh 250 --n_samples 8020

/home/victoria/miniconda3/envs/pointnet/lib/python3.10/site-packages/MDAnalysis/topology/LAMMPSParser.py:631: UserWarning: Guessed all Masses to 1.0
  warnings.warn('Guessed all Masses to 1.0')
/home/victoria/Documentos/ML/pre_entrenamiento.py:111: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  nlist = nsgrid.FastNS(args.cutoff*1.0,u.atoms.positions,ts.dimensions).search(u.atoms[sel].positions)
Total number of available samples for class lam: 8020
Total number of available samples for class lam_ord: 8020
Total number of available samples for class

In [ ]:
!python3 entrenamiento.py --dataset prueba_cutoff_3_maxneigh_250_scaled_shuffled_equal_samples.npy --labels prueba_cutoff_3_maxneigh_250_scaled_shuffled_equal_labels.npy  --batch_size 32 --nepochs 2

train shape: (15398, 250, 3, 1)
label shape: (15398, 3)
valid shape: (3850, 250, 3, 1)
label shape: (3850, 3)
Train on 15398 samples, validate on 3850 samples
Epoch 1/2
 7424/15398 [=============>................] - ETA: 5:14 - loss: 0.4260 - accuracy: 0.8161

In [23]:
from red_puntos import PointNet

In [6]:
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
import os
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.layers import Conv2D, MaxPooling2D

In [5]:
data = np.load('prueba_cutoff_3_maxneigh_250_scaled_shuffled_equal_samples.npy')
labels = np.load('prueba_cutoff_3_maxneigh_250_scaled_shuffled_equal_labels.npy')

In [21]:
labels.shape[-1]

3

In [6]:
a,b,c = data.shape
point_cloud_samples = data.reshape(a,b,c,-1)     

             
   # split data
full_train_samples,test_samples,full_train_labels,test_labels = train_test_split(point_cloud_samples, labels, test_size=0.2, random_state=13)
train_samples,valid_samples,train_labels,valid_labels = train_test_split(full_train_samples, full_train_labels, test_size=0.2, random_state=13)

print('train shape: ' + str(train_samples.shape))
print('label shape: ' + str(train_labels.shape))
print('valid shape: ' + str(valid_samples.shape))
print('label shape: ' + str(valid_labels.shape))

train shape: (15398, 250, 3, 1)
label shape: (15398, 3)
valid shape: (3850, 250, 3, 1)
label shape: (3850, 3)


In [24]:
train_samples[1].shape

(250, 3, 1)

In [7]:
net = PointNet(epochs=50,
              batch_size=16,
              lr=0.001)

In [8]:
red = net.defino_red(1e-5,0.3,3,(250, 3, 1))

In [9]:
red.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 250, 3, 64)        256       
                                                                 
 conv2d_1 (Conv2D)           (None, 250, 3, 64)        4160      
                                                                 
 conv2d_2 (Conv2D)           (None, 250, 3, 64)        4160      
                                                                 
 conv2d_3 (Conv2D)           (None, 250, 3, 128)       8320      
                                                                 
 conv2d_4 (Conv2D)           (None, 250, 3, 1024)      132096    
                                                                 
 max_pooling2d (MaxPooling2D  (None, 125, 2, 1024)     0         
 )                                                               
                                                        

In [10]:
red_entreno = red.entreno(train_samples, train_labels, valid_samples, valid_labels, 16, 5)

AttributeError: 'Sequential' object has no attribute 'entreno'

In [8]:
PointNet = Sequential()
PointNet.add(Conv2D(64, kernel_size=(1,3), activation= 'relu',
                             padding='same', 
                            #activity_regularizer=keras.regularizers.l2(1e-5),
                            kernel_regularizer=keras.regularizers.l2(1e-5),
                            input_shape = (250, 3, 1)))
PointNet.add(Conv2D(64, kernel_size=(1,1), activation= 'relu',
                            #is_training=is_training,
                            #activity_regularizer=keras.regularizers.l2(1e-5),
                            kernel_regularizer=keras.regularizers.l2(1e-5),
                            padding='same'))
PointNet.add(Conv2D(64, kernel_size=(1,1), activation= 'relu',
                            #is_training=is_training,
                            #activity_regularizer=keras.regularizers.l2(1e-5),
                            kernel_regularizer=keras.regularizers.l2(1e-5),
                            padding='same'))
PointNet.add(Conv2D(128, kernel_size=(1,1), activation= 'relu',
                           # is_training=is_training,
                            #activity_regularizer=keras.regularizers.l2(1e-5),
                            kernel_regularizer=keras.regularizers.l2(1e-5),
                            padding='same'))
PointNet.add(Conv2D(1024, kernel_size=(1,1), activation= 'relu',
                            #is_training=is_training,
                            #activity_regularizer=keras.regularizers.l2(1e-5),
                            kernel_regularizer=keras.regularizers.l2(1e-5),
                            padding='same'))

        # Symmetric function: max pooling
PointNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2),
                                 padding='same'))
        
        # MLP on global point cloud vector
        
PointNet.add(Flatten())
PointNet.add(Dense(512, activation='relu',#is_training=is_training,
                           #activity_regularizer=keras.regularizers.l2(1e-5),
                           kernel_regularizer=keras.regularizers.l2(1e-5)))
PointNet.add(Dense(256, activation= 'relu',#is_training=is_training,
                           #activity_regularizer=keras.regularizers.l2(1e-5),
                           kernel_regularizer=keras.regularizers.l2(1e-5)))
PointNet.add(Dropout(rate=0.3))
PointNet.add(Dense(3, activation='softmax'))

PointNet.compile(loss=keras.losses.categorical_crossentropy, optimizer= tf.keras.optimizers.Adam(),metrics=['accuracy'])


2022-08-26 09:59:25.113125: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 524288000 exceeds 10% of free system memory.
2022-08-26 09:59:25.423975: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 524288000 exceeds 10% of free system memory.


In [5]:
tf.config.list_physical_devices('GPU')

[]

In [9]:
PointNet.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 250, 3, 64)        256       
                                                                 
 conv2d_6 (Conv2D)           (None, 250, 3, 64)        4160      
                                                                 
 conv2d_7 (Conv2D)           (None, 250, 3, 64)        4160      
                                                                 
 conv2d_8 (Conv2D)           (None, 250, 3, 128)       8320      
                                                                 
 conv2d_9 (Conv2D)           (None, 250, 3, 1024)      132096    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 125, 2, 1024)     0         
 2D)                                                             
                                                      

In [10]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

early_stop = EarlyStopping(monitor='val_accuracy',
                           patience=10,
                           restore_best_weights=True,
                           mode='max')


In [11]:
PointNet_train = PointNet.fit(train_samples, train_labels, 
        callbacks=[early_stop], validation_data=(valid_samples, valid_labels))

482/482 [==============================] - 790s 2s/step - loss: 0.1354 - accuracy: 0.9573 - val_loss: 0.0962 - val_accuracy: 0.9771


In [9]:
red =  keras.models.load_model("pointnet_modelo_entrenado.h5") 

NameError: name 'keras' is not defined